In [10]:
import urllib.request
import pandas as pd
import zipfile
import geocoder as gcd



In [11]:
url = 'https://files.digital.nhs.uk/assets/ods/current/epraccur.zip'
filehandle, _ = urllib.request.urlretrieve(url)
zip_file_object = zipfile.ZipFile(filehandle, 'r')
first_file = zip_file_object.namelist()[0]
file = zip_file_object.open(first_file)
content = file.read()
csv_file = open('gp_data.csv', 'wb')
csv_file.write(content )
csv_file.close()
header_list = ["Organisation Code", 
"Name", 
"National Grouping", 
"High Level Health Geography", 
"Address line 1", 
"Address line 2", 
"Address line 3", 
"Address line 4", 
"Address line 5",
"Postcode",
"Open Date",
"Close Date",
"Status Code",
"Organisation Sub-Type Code",
"Commissioner",
"Join Provider/Purchaser Date",
"Left Provider/Purchaser Date",
"Contact Telephone Number",
"Null 1",
"Null 2",
"Null 3",
"Amended Record Indicator",
"Null 4",
"Provider/Purchaser",
"Null 5",
"Prescribing Setting",
"Null 6"
]
gp_practice_df = pd.read_csv('gp_data.csv', names=header_list)

In [12]:
gp_practice_df_1 = gp_practice_df.drop(columns = {
"High Level Health Geography", 
"Address line 1", 
"Address line 2", 
"Address line 3", 
"Address line 4", 
"Address line 5",
"Open Date",
"Close Date",
"Organisation Sub-Type Code",
"Commissioner",
"Join Provider/Purchaser Date",
"Left Provider/Purchaser Date",
"Contact Telephone Number",
"Null 1",
"Null 2",
"Null 3",
"Amended Record Indicator",
"Null 4",
"Provider/Purchaser",
"Null 5",
"Null 6"})
gp_practice_df_2 =  gp_practice_df_1[gp_practice_df_1["Status Code"] == "A"]
gp_practice_df_3 =  gp_practice_df_2[gp_practice_df_2["Prescribing Setting"] == 4]
gp_practice_df_eng = gp_practice_df_3[gp_practice_df_3["National Grouping"].str.contains("YAC|YAD|YAE|YAF|W00")==False]
gp_practice_df_eng_1 = gp_practice_df_eng.reset_index(drop = True)
gp_practice_df_eng_1

,Organisation Code,Name,National Grouping,Postcode,Status Code,Prescribing Setting
0,A81001,THE DENSHAM SURGERY,Y63,TS18 1HU,A,4
1,A81002,QUEENS PARK MEDICAL CENTRE,Y63,TS18 2AW,A,4
2,A81004,ACKLAM MEDICAL CENTRE,Y63,TS5 8SB,A,4
3,A81005,SPRINGWOOD SURGERY,Y63,TS14 7DJ,A,4
4,A81006,TENNANT STREET MEDICAL PRACTICE,Y63,TS18 2AT,A,4
...,...,...,...,...,...,...
6602,Y06810,WHITEHOUSE HEALTH CENTRE,Y61,MK8 1EQ,A,4
6603,Y06957,GARDEN PARK SURGERY,Y63,NE28 0PP,A,4
6604,Y07014,THE BERMUDA & MARLOWE PRACTICE,Y59,RG24 9DT,A,4
6605,Y07020,LAMBETH HEALTHCARE PRACTICE,Y56,SE21 8AU,A,4


In [13]:
def get_geocoder(postal_code_from_df):
  lat_lng_coords = None
  while(lat_lng_coords is None):
    g = gcd.google('{}, England, UK'.format(postal_code_from_df))
    lat_lng_coords = g.latlng
  latitude = lat_lng_coords[0]
  longitude = lat_lng_coords[1]
  return latitude,longitude

gp_practice_df_eng_1['Latitude'], gp_practice_df_eng_1['Longitude'] = get_geocoder(gp_practice_df_eng_1['Postcode'].values)
filename = 'gp_practice_df_eng.csv'
gp_practice_df_eng_1.to_csv(filename)